In [44]:
import grpc
import numpy as np
import cv2
from tensorflow import make_tensor_proto, make_ndarray
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

# Define ImageNet class labels (simplified example)
imagenet_classes = {
    0: "tench",
    1: "goldfish",
    340: "zebra",
    644: "match",
    # Add more labels as needed...
}


In [ ]:
# Preprocess the image
def preprocess_image(image_path, size=224):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found at {image_path}")
    img = cv2.resize(img, (size, size))
    img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
    img = img.transpose(2, 0, 1)  # HWC to CHW
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img


# Run inference and process the output
def infer_image(image_path, grpc_address, grpc_port, model_name, input_name, output_name):
    # Create gRPC channel and stub
    channel = grpc.insecure_channel(f"{grpc_address}:{grpc_port}")
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

    # Preprocess the image
    input_data = preprocess_image(image_path)
    print(f"Input shape: {input_data.shape}, dtype: {input_data.dtype}, range: {np.min(input_data)}-{np.max(input_data)}")

    # Create gRPC PredictRequest
    request = predict_pb2.PredictRequest()
    request.model_spec.name = model_name
    request.inputs[input_name].CopyFrom(make_tensor_proto(input_data, shape=input_data.shape))

    # Send request and get response
    response = stub.Predict(request, timeout=10.0)

    # Extract the tensor content
    output = make_ndarray(response.outputs[output_name])
    predicted_class = np.argmax(output)

    # Map class index to label
    label = imagenet_classes.get(predicted_class, "Unknown")
    return predicted_class, label


In [48]:

if __name__ == "__main__":
    # Parameters
    grpc_address = "pecosv"
    grpc_port = 9000
    model_name = "resnet"
    input_name = "0"  # Input tensor name from logs
    output_name = "prob"  # Confirm with model metadata
    image_path = "C:/Users/pedro/Documents/developer/model-server/assets/zebra.jpeg"

    try:
        # Run inference
        predicted_class, label = infer_image(image_path, grpc_address, grpc_port, model_name, input_name, output_name)

        # Print results
        print(f"Predicted Class Index: {predicted_class}")
        print(f"Predicted Label: {label}")
    except Exception as e:
        print(f"Error during inference: {e}")


Input shape: (1, 3, 224, 224), dtype: float32, range: 0.0-1.0
Error during inference: Cannot convert the argument `type_value`: 0 to a TensorFlow DType.


In [ ]:
import grpc
import numpy as np
import cv2
from tensorflow import make_tensor_proto, make_ndarray
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

# Define ImageNet class labels (simplified, replace with full mapping)
imagenet_classes = {
    0: "tench",
    1: "goldfish",
    2: "great white shark",
    340: "zebra",  # Include relevant classes for simplicity
    # Add more classes as needed...
}

# Preprocess the image
def preprocess_image(image_path, size=224):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found at {image_path}")
    img = cv2.resize(img, (size, size))
    img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
    img = img.transpose(2, 0, 1)  # HWC to CHW
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Run inference and process the output
def infer_image(image_path, grpc_address, grpc_port, model_name, input_name, output_name):
    # Create gRPC channel and stub
    channel = grpc.insecure_channel(f"{grpc_address}:{grpc_port}")
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

    # Preprocess the image
    input_data = preprocess_image(image_path)

    # Create gRPC PredictRequest
    request = predict_pb2.PredictRequest()
    request.model_spec.name = model_name
    request.inputs[input_name].CopyFrom(
    make_tensor_proto(input_data, dtype=np.float32, shape=input_data.shape)
)


    # Send request and get response
    response = stub.Predict(request, timeout=10.0)
    print(f"Input data type: {input_data.dtype}, shape: {input_data.shape}, range: {input_data.min()} - {input_data.max()}")

    # Extract the tensor content
    output = make_ndarray(response.outputs[output_name])
    predicted_class = np.argmax(output)

    # Map class index to label
    label = imagenet_classes.get(predicted_class, "Unknown")
    return predicted_class, label

if __name__ == "__main__":
    # Parameters
    grpc_address = "pecosv"
    grpc_port = 9000
    model_name = "resnet"
    input_name = "0"
    output_name = "1463"

    image_path = "C:/Users/pedro/Documents/developer/model-server/assets/zebra.jpeg"

    try:
        # Run inference
        predicted_class, label = infer_image(image_path, grpc_address, grpc_port, model_name, input_name, output_name)

        # Print results
        print(f"Predicted Class Index: {predicted_class}")
        print(f"Predicted Label: {label}")
    except Exception as e:
        print(f"Error during inference: {e}")


Input data type: float32, shape: (1, 3, 224, 224), range: 0.0 - 1.0
Error during inference: Cannot convert the argument `type_value`: 0 to a TensorFlow DType.


In [61]:
import requests

url = "http://pecosv:9001/v1/models/resnet/metadata"
response = requests.get(url)
print(response.json())


{'modelSpec': {'name': 'resnet', 'signatureName': '', 'version': '1'}, 'metadata': {'signature_def': {'@type': 'type.googleapis.com/tensorflow.serving.SignatureDefMap', 'signatureDef': {'serving_default': {'inputs': {'0': {'dtype': 'DT_FLOAT', 'tensorShape': {'dim': [{'size': '1', 'name': ''}, {'size': '3', 'name': ''}, {'size': '224', 'name': ''}, {'size': '224', 'name': ''}], 'unknownRank': False}, 'name': '0'}}, 'outputs': {'1463': {'dtype': 'DT_FLOAT', 'tensorShape': {'dim': [{'size': '1', 'name': ''}, {'size': '1000', 'name': ''}], 'unknownRank': False}, 'name': '1463'}}, 'methodName': '', 'defaults': {}}}}}}
